В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [20]:
dataname = 'default_ON_QnSC_100k'
dataname_initial = 'default'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_100k'

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [21]:
model_save_path_hist = []
idx = [5, 6]

for sigma in [0.01, 0.25]:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [15]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/default_ON_QnSC_100k` folder


In [16]:
normalizers

{'len_num_prev': 14,
 'len_cat_prev': 77,
 'len_target_prev': 1,
 'num_normalizer': QuantileTransformer(n_quantiles=900, output_distribution='normal',
                     random_state=0, subsample=1000000000),
 'cat_normalizer': StandardScaler()}

### Sample + Eval

In [22]:
CONFIG.get_all_args()

{'dataname': 'default_ON_QnSC_100k',
 'method': 'tabddpm_ON_QnSC_100k_const_0_100k_const_0_01',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': './synthetic/default_ON_QnSC_100k/initial_tabddpm_ON_QnSC_100k_const_0_100k_const_0_01.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 1,
 'real_path': './synthetic/default_ON_QnSC_100k/initial_real.csv',
 'test_path': './synthetic/default_ON_QnSC_100k/initial_test.csv',
 'info_path': './data/default_ON_QnSC_100k/initial_info.json',
 'save_path': './synthetic/magic_ON_QnSC_100k/initial_tabddpm_ON_QnSC_100k_mult_6.csv',
 'num_clusters': 25,
 'save_cat': None}

In [18]:
model_save_path_hist

['./models/tabddpm_ON_QnSC_100k_const_0/ckpt/default/tabddpm_ON_QnSC_100k_const_0_100k_const_0_01',
 './models/tabddpm_ON_QnSC_100k_const_0/ckpt/default/tabddpm_ON_QnSC_100k_const_0_100k_const_0_25',
 './models/tabddpm_ON_QnSC_100k_const_0/ckpt/default/tabddpm_ON_QnSC_100k_const_0_100k_mult_5',
 './models/tabddpm_ON_QnSC_100k_const_0/ckpt/default/tabddpm_ON_QnSC_100k_const_0_100k_mult_6']

In [23]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False) # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TABDDPM_ON_QNSC_100K_CONST_0_01: DEFAULT_ON_QNSC_100K Dataset
Original Logistic: 0.467
Synthetic Logistic: 0.469
Original Tree: 0.480
Synthetic Tree: 0.445 

TABDDPM_ON_QNSC_100K_CONST_0_01 Accuracy Loss: DEFAULT_ON_QNSC_100K
Logistic: -0.522%
Tree: 7.296% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 24/24 [00:00<00:00, 55.48it/s]|
Column Shapes Score: 98.58%

(2/2) Evaluating Column Pair Trends: |██████████| 276/276 [00:01<00:00, 139.71it/s]|
Column Pair Trends Score: 97.79%

Overall Score (Average): 98.18%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_100K_CONST_0_01: 1.419 ± 0.568
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_100K_CONST_0_01: 2.213 ± 1.375


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:12<00:00,  2.96it/s]


...

TABDDPM_ON_QNSC_100K_MULT_6: DEFAULT_ON_QNSC_100K Dataset
Original Logistic: 0.467
Synthetic Logistic: 0.466
Original Tree: 0.480
Synthetic Tree: 0.433 

TABDDPM_ON_QNSC_100K_MULT_6 Accuracy Loss: DEFAULT_ON_QNSC_100K
Logistic: 0.299%
Tree: 9.649% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 24/24 [00:00<00:00, 61.20it/s]|
Column Shapes Score: 98.38%

(2/2) Evaluating Column Pair Trends: |██████████| 276/276 [00:01<00:00, 147.50it/s]|
Column Pair Trends Score: 97.5%

Overall Score (Average): 97.94%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_100K_MULT_6: 1.617 ± 0.618
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_100K_MULT_6: 2.496 ± 1.953


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:12<00:00,  2.93it/s]


Saving scores to  eval/mle/default_ON_QnSC_100k/tabddpm_ON_QnSC_100k_mult_6.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_100K_MULT_6: 0.730 ± 0.013
default_ON_QnSC_100k, tabddpm_ON_QnSC_100k_mult_6: 0.9720922839506173
n_clusters: 25
n_clusters: 25
n_clusters: 25
n_clusters: 25
n_clusters: 25
DCR Score, a value closer to 0.5 is better
default_ON_QnSC_100k-tabddpm_ON_QnSC_100k_mult_6, DCR Score = 0.5183333333333333
=========== All Features ===========
Data shape:  (27000, 93)
alpha precision: 0.968760, beta recall: 0.470020


In [24]:
final_metrics_table = []
model_short = 'tabddpm_ON_QnSC_100k'
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [25]:
final_metrics_table#.drop(columns=('base_metrics', 0))

\
                  Model                             Type   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_01   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_25   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_5   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_6   

                             base_metrics                                   \
                   Data Original Logistic Synthetic Logistic Original Tree   
0  default_ON_QnSC_100k           0.46697           0.469409      0.479705   
0  default_ON_QnSC_100k           0.46697           0.460568      0.479705   
0  default_ON_QnSC_100k           0.46697           0.468915      0.479705   
0  default_ON_QnSC_100k           0.46697           0.465574      0.479705   

                                                                  \
  Synthetic Tree Accuracy Loss Logistic, % Accuracy Loss Tree, %   
0       0.444707                 -0.522281              7.295638   
0       0.450755                  1.371086              6.034859   
0       0.446035                 -0.416356              7.018944   
0       0.433417                  0.299080              9.649208   

              similarity  ...  \
  Column Shapes Score, %  ...   
0              98.581019  ...   
0              98.228858  ...   
0              98.338580  ...   
0              98.383179  ...   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                            1.418981   
0                                            1.771142   
0                                            1.661420   
0                                            1.616821   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                0.567995   
0                                                0.961256   
0                                                0.651641   
0                                                0.618087   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                2.212638   
0                                                2.509200   
0                                                2.043152   
0                                                2.496215   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                    1.374723   
0                                                    1.442332   
0                                                    1.253390   
0                                                    1.953409   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                   0.703890   
0                                   0.704959   
0                                   0.706248   
0                                   0.730075   

                                                  detection       DCR  \
  ROC - AUC обучения на синтетических данных, std     Score     Score   
0                                        0.011636  0.966299  0.521185   
0                                        0.014833  0.981919  0.515889   
0                                        0.013301  0.975856  0.517111   
0                                        0.012778  0.972092  0.518333   

          quality              
  alpha precision beta recall  
0        0.976261    0.466869  
0        0.970803    0.317462  
0        0.968370    0.461743  
0        0.968760    0.470020  

[4 rows x 22 columns]

In [27]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
                  Model                             Type   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_01   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_5   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_6   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_25   

                             base_metrics                                   \
                   Data Original Logistic Synthetic Logistic Original Tree   
0  default_ON_QnSC_100k             0.467             0.4694        0.4797   
0  default_ON_QnSC_100k             0.467             0.4689        0.4797   
0  default_ON_QnSC_100k             0.467             0.4656        0.4797   
0  default_ON_QnSC_100k             0.467             0.4606        0.4797   

                                                                  \
  Synthetic Tree Accuracy Loss Logistic, % Accuracy Loss Tree, %   
0         0.4447                   -0.5223                7.2956   
0         0.4460                   -0.4164                7.0189   
0         0.4334                    0.2991                9.6492   
0         0.4508                    1.3711                6.0349   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                98.5810                     97.7874   
0                98.3386                     97.9568   
0                98.3832                     97.5038   
0                98.2289                     97.4908   

                              \
  Overall Score (Average), %   
0                    98.1842   
0                    98.1477   
0                    97.9435   
0                    97.8598   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              1.4190   
0                                              1.6614   
0                                              1.6168   
0                                              1.7711   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  0.5680   
0                                                  0.6516   
0                                                  0.6181   
0                                                  0.9613   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                  2.2126   
0                                                  2.0432   
0                                                  2.4962   
0                                                  2.5092   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                      1.3747   
0                                                      1.2534   
0                                                      1.9534   
0                                                      1.4423   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                     0.7039   
0                                     0.7062   
0                                     0.7301   
0                                     0.7050   

                                                  detection     DCR  \
  ROC - AUC обучения на синтетических данных, std     Score   Score   
0                                          0.0116    0.9663  0.5212   
0                                          0.0133    0.9759  0.5171   
0                                          0.0128    0.9721  0.5183   
0                                          0.0148    0.9819  0.5159   

          quality              
  alpha precision beta recall  
0          0.9763      0.4669  
0        

In [20]:
f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv'

'./eval/total/default_ON_QnSC_100k/tabddpm_ON_QnSC_100k_final_metrcs_table.csv'

In [21]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)